In [98]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from PIL import Image
import pickle

#sys.path.append(os.pardir) # 为了导入父目录中的文件而进行的设定

def AND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.7
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1
    
#阶跃函数
def step_function(x):
    return np.array(x > 0, dtype=int)

#sigmoid函数 f(x) = 1 / (1 + e ** -x)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

#ReLU
def ReLU(x):
    return np.maximum(0, x)

#激活函数
def identity_function(x):
    return x

#定义softmax函数
#def softmax(x):
#    return np.exp(x) / np.exp(x).sum()

#改进softmax函数
def softmax(x):
    dim = x.shape[0]
    if x.ndim == 1:
        x_max = np.max(x)
        return np.exp(x - x_max) / np.exp(x - x_max).sum()
    if x.ndim == 2:
        x_max = np.max(x, axis = 1)
        return np.exp(x - x_max.reshape(dim, -1)) / np.exp(x - x_max.reshape(dim, -1)).sum(axis = 1).reshape(dim, -1)

#x = np.arange(-5, 5, 0.1)
#plt.plot(x, sigmoid(x), label = 'sigmoid')
#plt.plot(x, step_function(x), linestyle = '--', label = 'step_function')
#plt.plot(x, ReLU(x), linestyle = '-', label = 'ReLU')
#plt.xlabel("x") # x轴标签
#plt.ylabel("y") # y轴标签
#plt.title('sin & cos') # 标题
#plt.legend()
##plt.ylim(0, 1)
#plt.show()

#x = np.arange(-5, 5, 0.1)
#plt.plot(x, ReLU(x), linestyle = '-')
#plt.show()

# 输出层的激活函数用σ() 表示，不同于隐藏层的激活函数h()（σ读作sigma）。
#X = np.array([1.0, 0.5])
#W1 = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
#B1 = np.array([0.1, 0.2, 0.3])
#A1 = np.dot(X, W1) + B1
#Z1 = sigmoid(A1)
#W2 = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
#B2 = np.array([0.1, 0.2])
#A2 = np.dot(Z1, W2) + B2
#Z2 = sigmoid(A2)
#W3 = np.array([[0.1, 0.3], [0.2, 0.4]])
#B3 = np.array([0.1, 0.2])
#A3 = np.dot(Z2, W3) + B3
#Y = identity_function(A3)


#人工定义神经网络初始化参数
def init_network():
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1, 0.3], [0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])
    return network

def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    Z1 = sigmoid(np.dot(x, W1) + b1)
    Z2 = sigmoid(np.dot(Z1, W2) + b2)
    y = identity_function(np.dot(Z2, W3) + b3)
    return y

#x = np.array([1.0, 0.5])
#y = forward(init_network(), x)


def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
    
#显示图片和表情
#img = x_train[0]
#label = t_train[0]
#img = img.reshape(28, -1)
#print(label)
#img.shape
#img_show(img)

def get_data():
    (x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open('.\\ch03\\sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    return network

#随机梯度下降
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    Z1 = sigmoid(np.dot(x, W1) + b1)
    Z2 = sigmoid(np.dot(Z1, W2) + b2)
    y = softmax(np.dot(Z2, W3) + b3)
    return y

#x, t = get_data()
#network = init_network()
#accuracy_cnt = 0
#for i in range(len(x)):
#    y = predict(network, x[i])
#    p = np.argmax(y) # 获取概率最高的元素的索引
#    if p == t[i]:
#        accuracy_cnt += 1
#print("Accuracy:" + str(float(accuracy_cnt) / len(x)))


#批量梯度下降（batch）
x, t = get_data()
network = init_network()
batch_size = 100
accuracy_cnt = 0
for i in range(0, len(x), batch_size):
    x_batch = x[i: i + batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis = 1)
    accuracy_cnt += np.sum(p == t[i: i + batch_size])

#均方误差，输入numpy数组
def mean_squared_error(y_pre, y):
    return 0.5 * np.sum((y - y_pre) ** 2)

#交叉熵，输入numpy数组
def cross_entropy(y_pre, y):
    delta = 1e-7
    return - np.sum(y_pre * np.log(y + delta))

#mini-batch算法，每次选出一小部分作为整体的平均代表
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

#改良交叉熵
def cross_entropy_2(y_pre, y):
    if y.ndim != 1:
        y = y.reshape(-1)
        y_pre = y_pre.reshape(-1)
    delta = 1e-7
    return (-np.sum(y_pre * np.log(y + delta))) / y.shape[0]

#前向差分
def num_differential(f, x):
    delta_x = 1e-20
    return (f(x + delta_x) - f(x)) / delta_x

#中心差分
def num_center_differential(f, x):
    h = 1e-10
    return (f(x + h) - f(x - h)) / (2 * h)

def fn(x):
    return 0.01 * x * x + 0.1 * x

def line(f, x):
    y_diff = num_center_differential(f, x)
    y_0 = f(x) - y_diff * x
    return lambda t: y_diff * t + y_0

#微分图
#x = np.arange(0, 20, 0.1)
#y = fn(x)
#plt.plot(x, y)
#plt.xlabel('x')
#plt.ylabel('f(x)')
#lf = line(fn, 10)
#y2 = lf(x)
#plt.plot(x, y2)
#plt.show()

#偏导数实现
def fn_2(x):
    return np.sum(x ** 2)

def fn_partial1(x):
    return x * x + 4 * 4

def fn_partial2(x):
    return 3 * 3 + x * x

#梯度实现
def num_gradient(f, x):
    #h = 1e-4
    grad = np.zeros_like(x)
    for i in range(grad.size):
        grad[i] = num_center_differential(f, x[i])
    return grad

#num_gradient(fn_2, np.array([3, 0, 6]))

#梯度下降实现
def gradient_descent(loss, x_init, lr = 0.01, step_num = 100):
    x = x_init
    for i in range(step_num):
        grad = num_gradient(loss, x)
        x = x - lr * grad
    return x

#实例
x_init = np.array([-3, 4])
gradient_descent(fn_2, x_init, lr = 0.2)

#构建神经网络类
class SimpleNet:
    def __init__(self):
        self.w = np.random.randn(2, 3)
    
    def predict(self, x):
        return np.dot(x, self.w)
    
    def loss(self, x, t):
        y = softmax(self.predict(x))
        loss = cross_entropy_2(y, t)
        return loss
    
    def num_gradient(self, x, t):
        return 

    
#测试
net = SimpleNet()
x = np.array([0.6, 0.9])
p = net.predict(x)
t = np.array([0, 0, 1])
net.loss(x, t)
def f(W):
    return net.loss(x, t)
num_gradient(f, net.w)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [109]:
a = np.arange(6).reshape(2, 3)  


[0 1 2]
[3 4 5]
